In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import grangercausalitytests
import statsmodels.api as sm
import warnings

# This will suppress ALL FutureWarning messages
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)
from Functions import read_data, load_additional_data, extract_granger_results, granger_results, granger_effect_direction

In [7]:
import glob
folder_path = "Additional Data/*.csv"
files = glob.glob(folder_path)
data_df = read_data()
data_df = load_additional_data(data_df, files)
cov_matrix = data_df.cov()
corr_matrix = data_df.corr()

Additional Data\Auto Inventory Sales Ratio - SA.csv
Additional Data\CUSR0000SETA02.csv
Additional Data\CUUR0000SETA01.csv
Additional Data\DAUTOSA (1).csv
Additional Data\Domestic Auto Production - SA.csv
Additional Data\Domestic Auto Sales- SAAR.csv
Additional Data\FEDFUNDS.csv
Additional Data\Foreign Auto Sales - SA.csv
Additional Data\Manufacturers New Order Vehicles and Parts - SA.csv
Additional Data\Natural Gas Consumption - SA.csv
Additional Data\Natural Gas Conventional Price.csv
Additional Data\Public Transit Ridership.csv
Additional Data\TERMCBAUTO48NS.csv
Additional Data\UMCSENT.csv
Additional Data\Vehicle Miles Traveled.csv


In [9]:
df_diff = data_df.diff().dropna()
display(df_diff.head())

,Total Sales,New Orders,AISRSA,CPI_Used,CPI_NewVehicle,DAUTOSA,DAUPSA,DAUTOSAAR,FEDFUNDS,FAUTOSA,AMVPNO,GASREGCOVM,NATURALGASD11,TRANSIT,TERMCBAUTO48NS,UMCSENT,VMT
DATE,,,,,,,,,,,,,,,,,
2000-02-01,0.766,-3860.0,-0.168,-0.9,-0.3,18.2,-8.1,0.219,0.28,3.4,-3860.0,0.092,29.7,31602.0,0.00,-0.7,-4181.0
2000-03-01,-1.058,941.0,0.148,0.0,0.3,-41.7,1.1,-0.501,0.12,-7.9,941.0,0.127,-120.2,87194.0,0.00,-4.2,33229.0
2000-04-01,-0.404,-105.0,0.070,1.0,0.2,-10.7,13.1,-0.129,0.17,5.1,-105.0,-0.061,120.3,-86272.0,0.00,2.1,-4792.0
2000-05-01,0.004,-1914.0,0.012,1.4,-0.2,-2.5,5.8,-0.030,0.25,0.3,-1914.0,0.019,129.8,60788.0,0.34,1.5,14803.0
2000-06-01,-0.347,1185.0,0.085,0.3,-0.4,-10.2,-17.2,-0.122,0.26,-0.2,1185.0,0.156,-89.7,-24335.0,0.00,-4.3,462.0


Granger Casuality

In [10]:
granger_data = df_diff[['New Orders', 'NATURALGASD11']]

result_cols = ['Total Sales', 'New Orders']
exog_cols = [col for col in df_diff.columns if col not in result_cols]

granger_results = []
for result_col in result_cols:
    for exog_col in exog_cols:
        granger_data =df_diff[[result_col, exog_col]]
        granger_results += extract_granger_results(granger_data, 5)

granger_results_df = pd.DataFrame(granger_results)
display(granger_results_df[granger_results_df['decision']=='Reject H0 (Granger Causality)'])

,result,exog,lag,p_value,decision
15,Total Sales,DAUTOSA,1,4.559607e-02,Reject H0 (Granger Causality)
16,Total Sales,DAUTOSA,2,4.207102e-02,Reject H0 (Granger Causality)
17,Total Sales,DAUTOSA,3,3.040235e-02,Reject H0 (Granger Causality)
18,Total Sales,DAUTOSA,4,4.615245e-02,Reject H0 (Granger Causality)
19,Total Sales,DAUTOSA,5,3.248113e-02,Reject H0 (Granger Causality)
25,Total Sales,DAUTOSAAR,1,4.535178e-02,Reject H0 (Granger Causality)
26,Total Sales,DAUTOSAAR,2,4.195274e-02,Reject H0 (Granger Causality)
27,Total Sales,DAUTOSAAR,3,3.012235e-02,Reject H0 (Granger Causality)
28,Total Sales,DAUTOSAAR,4,4.597130e-02,Reject H0 (Granger Causality)
29,Total Sales,DAUTOSAAR,5,3.235682e-02,Reject H0 (Granger Causality)


In [12]:
causal_df =  granger_results_df[granger_results_df['decision'] == 'Reject H0 (Granger Causality)']
causal_pairs = list(zip(causal_df['exog'],causal_df['result'], causal_df['lag']))
causal_df = granger_effect_direction(df_diff, causal_pairs)
display(causal_df)

,X,Y,lag,coef,direction
0,DAUTOSA,Total Sales,1,[-0.007321321700016044],negative
1,DAUTOSA,Total Sales,2,[-0.004937097273083017],negative
2,DAUTOSA,Total Sales,3,[-0.0010389830376012436],negative
3,DAUTOSA,Total Sales,4,[0.000242936972431301],positive
4,DAUTOSA,Total Sales,5,[0.0024323460925593787],positive
5,DAUTOSAAR,Total Sales,1,[-0.6103332769695264],negative
6,DAUTOSAAR,Total Sales,2,[-0.41142309358629575],negative
7,DAUTOSAAR,Total Sales,3,[-0.08675261588703909],negative
8,DAUTOSAAR,Total Sales,4,[0.020989658107877857],positive
9,DAUTOSAAR,Total Sales,5,[0.20238680159636135],positive
